# Solving Linear Systems with Matrix Decomposition

선형 시스템 `Ax=b`를 푸는 것은 과학과 공학의 여러 분야에서 가장 근본적인 문제 중 하나입니다.
본 실습에서는 행렬 분해(Matrix Decomposition)가 어떻게 이러한 시스템을 효율적이고 안정적으로 해결하는지 알아봅니다.

크게 두 가지 시나리오를 다룹니다:
1.  **해가 유일하게 존재하는 경우**: Cholesky 분해를 사용하여 정확한 해를 구합니다.
2.  **해가 존재하지 않는 경우**: QR 분해를 사용하여 오차를 최소화하는 최적의 근사해(최소제곱 해)를 구합니다.

### 준비하기: 라이브러리 설치 및 불러오기

    본 실습에서는 `numpy`, `matplotlib`과 함께 과학 계산 라이브러리인 `scipy`를 사용합니다.
    아래 코드는 라이브러리들이 설치되어 있는지 확인하고, 만약 없다면 자동으로 설치합니다.

In [ ]:
# 라이브러리 설치
!pip install numpy matplotlib scipy

In [ ]:
# 라이브러리 불러오기
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg

# 재현성을 위해 랜덤 시드 고정
np.random.seed(0)

---
## Part 1: Cholesky 분해를 이용한 선형 시스템 풀이 (정확한 해)
---
선형 시스템 `Ax=b`에서 행렬 `A`가 대칭(Symmetric)이면서 양의 정부호(Positive Definite)인 특별한 조건을 만족할 때, Cholesky 분해를 사용하면 매우 효율적으로 해 `x`를 구할 수 있습니다.

#### Cholesky 분해의 원리
- 행렬 `A`를 하삼각행렬(Lower-triangular matrix) `L`과 그 전치 행렬 `L.T`의 곱으로 분해합니다: `A = L @ L.T`
- 이를 원래의 선형 시스템에 대입하면 `(L @ L.T) @ x = b`가 됩니다.
- 문제는 두 개의 더 간단한 선형 시스템으로 나뉩니다.
  1. `L @ y = b` 를 풀어 중간 벡터 `y`를 구합니다. (전방 대입법)
  2. `L.T @ x = y` 를 풀어 최종 해 `x`를 구합니다. (후방 대입법)

이 방식은 역행렬을 직접 구하는 것보다 수치적으로 훨씬 안정적이고 계산 비용이 저렴합니다.

### 1.1. Cholesky 분해 실습
`scipy.linalg.cholesky`를 사용하여 `A`를 분해하고, `scipy.linalg.solve_triangular`를 두 번 사용하여 해 `x`를 구합니다.

In [ ]:
# 실습: 아래 행렬 A와 벡터 b의 값을 직접 바꿔보며(A는 대칭 유지), 해가 어떻게 변하는지 확인해보세요.
#      A가 양의 정부호가 아니게 되면(예: A[0,0] = -1) 어떤 오류가 발생하는지도 관찰해보세요.
A = np.array([
    [4., 2., 0.],
    [2., 5., 2.],
    [0., 2., 5.]
])
# 이 시스템의 해는 x = [1, 2, 3] 입니다. b는 A @ x의 결과입니다.
b = np.array([8., 18., 19.])

print("행렬 A (대칭, 양의 정부호):\n", A)
print("\n벡터 b:\n", b)

# 1. Cholesky 분해: A = L @ L.T
#    scipy.linalg.cholesky는 기본적으로 하삼각행렬 L을 반환합니다.
try:
    L = scipy.linalg.cholesky(A, lower=True)
    print("\n하삼각행렬 L:\n", np.round(L, 2))

    # 2. 첫 번째 시스템 풀이: L @ y = b
    y = scipy.linalg.solve_triangular(L, b, lower=True)
    print("\n중간 해 y:\n", np.round(y, 2))

    # 3. 두 번째 시스템 풀이: L.T @ x = y
    x = scipy.linalg.solve_triangular(L.T, y, lower=False)
    print("\n최종 해 x (Cholesky):\n", np.round(x, 2))

    # 4. 검증: np.linalg.solve와 결과 비교
    x_direct = np.linalg.solve(A, b)
    print("\n최종 해 x (np.linalg.solve):\n", np.round(x_direct, 2))
    print("\n두 방법의 해가 일치하는가?", np.allclose(x, x_direct))

except np.linalg.LinAlgError:
    print("오류: 행렬이 대칭, 양의 정부호가 아닙니다.")

---
## Part 2: QR 분해를 이용한 최소제곱 문제 풀이 (근사해)
---
데이터 분석에서 마주하는 많은 문제는 `Ax=b` 형태에서 행렬 `A`의 행의 개수가 열의 개수보다 많은 **Over-determined system**입니다.
이러한 시스템은 일반적으로 정확한 해가 존재하지 않습니다. 대신, 우리는 잔차(residual)의 제곱합 `||Ax - b||²`를 최소화하는 **근사해(approximate solution)** `x̂`를 찾고자 합니다. 이를 **최소제곱(Least Squares) 문제**라고 합니다.

#### QR 분해의 원리
- QR 분해는 행렬 `A`를 직교행렬(Orthogonal matrix) `Q`와 상삼각행렬(Upper-triangular matrix) `R`의 곱으로 표현합니다: `A = Q @ R`
- 이를 최소제곱 문제에 적용하면, 문제는 `R @ x̂ = Q.T @ b` 라는 간단한 선형 시스템으로 변환됩니다.
- `R`이 상삼각행렬이므로, 이 시스템은 후방 대입법으로 쉽게 풀 수 있습니다.

QR 분해는 최소제곱 문제를 푸는 매우 안정적이고 신뢰성 높은 방법입니다.

### 응용 예제: 2D 데이터 직선 피팅 (Straight-line Fit)
산점도 데이터를 가장 잘 설명하는 직선 `y = c₀ + c₁x`를 찾는 것은 대표적인 최소제곱 문제입니다.
`N`개의 데이터 포인트 `(xᵢ, yᵢ)`가 주어졌을 때, 우리는 다음 식의 오차를 최소화하는 파라미터 `c = [c₀, c₁]`를 찾아야 합니다.

`|| A @ c - y ||²`

여기서,
- `y`는 `[y₁, y₂, ..., yₙ]` 벡터입니다.
- `c`는 `[c₀, c₁]` 벡터 (우리가 찾고자 하는 직선의 절편과 기울기)입니다.
- `A`는 `N x 2` 크기의 설계 행렬(Design Matrix)로, 첫 번째 열은 1(절편용), 두 번째 열은 x 데이터로 구성됩니다.
  ```
      [ 1  x₁ ]
  A = [ 1  x₂ ]
      [ :  :  ]
      [ 1  xₙ ]
  ```

### 2.1. 최소제곱 직선 피팅 실습
QR 분해를 사용하여 주어진 데이터에 가장 잘 맞는 직선의 파라미터를 찾고, 결과를 시각화합니다.

In [ ]:
# 실습: true_c0, true_c1 값을 바꾸거나, 노이즈의 강도(아래 normal 함수의 세 번째 인자)를 바꿔보며 직선 피팅이 어떻게 변하는지 관찰해보세요.
# 1. 직선 피팅을 위한 간단한 데이터 생성
true_c0 = 2.0  # y-절편
true_c1 = 3.0  # 기울기

# 실습: 생성할 데이터 포인트 개수를 바꿔보세요.
num_data_points = 6
x_data = np.arange(num_data_points)
# x_data = np.array([0, 1, 2, 3, 4, 5])

noise = np.random.normal(0, 1.0, size=x_data.shape)
y_data = true_c0 + true_c1 * x_data + noise

# 2. 최소제곱 문제를 위한 설계 행렬 A와 벡터 y 구성
#    np.vstack([...]).T 또는 np.c_[...]를 사용하여 열을 합칩니다.
A_fit = np.c_[np.ones(x_data.shape[0]), x_data]
print("설계 행렬 A의 크기:", A_fit.shape)

# 3. QR 분해 수행
Q, R = np.linalg.qr(A_fit)
print("Q 행렬의 크기:", Q.shape)
print("R 행렬의 크기:", R.shape)

# 4. R @ c_hat = Q.T @ y_data 풀기
QTb = Q.T @ y_data
c_hat = scipy.linalg.solve_triangular(R, QTb, lower=False)

print("\n--- 찾은 파라미터 vs 실제 파라미터 ---")
print(f"           | {'찾은 값':^7s} | {'실제 값':^7s}")
print("------------------------------------")
print(f"절편 (c₀)  | {c_hat[0]:^7.2f} | {true_c0:^7.2f}")
print(f"기울기 (c₁) | {c_hat[1]:^7.2f} | {true_c1:^7.2f}")

# 5. 결과 시각화
plt.figure(figsize=(10, 6))
plt.scatter(x_data, y_data, label='Data Points', alpha=0.7)
plt.plot(x_data, c_hat[0] + c_hat[1] * x_data, color='red', linewidth=2, label='Fitted Line (Least Squares)')
plt.title('Straight-line Fit using QR Decomposition')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.grid(True)
plt.show() 